In [1]:
import numpy as np
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

path_data = '../harvey-twitter/data/harvey_twitter_dataset/02_archive_only/HurricaneHarvey.json'
#path_data = "/corral-repl/utexas/Trump-Tweets/Harvey/unt_data/harvey_twitter_dataset/02_archive_only/HurricaneHarvey.json"

[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('data/twitter_text.csv',lineterminator='\n')
df.head()

,Unnamed: 0,text
0,0,RT @aaronjayjack: Displaced dog jumped into my...
1,1,RT @JanelleKHOU: Water is seeping into the stu...
2,2,RT @ReadyHarris: OPEN SHELTER: North Shore 9th...
3,3,RT @LHuffmanPowell: This dog is walking around...
4,4,RT @fema: Please remember: #Harvey is still an...


In [3]:
def process_tweet(tweet):
    st_1 = []
    for w in tweet.split():
        #remove retweet annotation if present:
        if w == 'RT' or w[0] == '@':
            continue
        #remove hashtag symbol
        elif w[0] == '#':
            st_1.append(w[1:])
        #replace link with LINK keyword
        elif w[:4] == 'http':
            st_1.append('link')
        else:
            st_1.append(w)
    
    st_2 =  word_tokenize(' '.join(st_1))
    
    #remove stop words and punctuation, make everything lowercase
    st_3 = [w.lower() for w in st_2 if w.isalpha() and not w in stop_words]
    
    #lemmatization (converts all words to root form for standardization)
    lem = WordNetLemmatizer()
    st_4 = list(map(lambda x: lem.lemmatize(x, pos='v'), st_3))
    return st_4

doc_sample = df['text'][0]
process_tweet(doc_sample)

['displace',
 'dog',
 'jump',
 'jeep',
 'please',
 'share',
 'help',
 'find',
 'owner',
 'harvey',
 'hurricane',
 'displacedpets',
 'link']

In [6]:
#for backwards-compatibility (Python 3.7 has str.isascii)
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

tweets = df['text']
processed_tweets = tweets.map(process_tweet)
processed_tweets.to_csv('data/twitter_text_processed.csv')
processed_tweets.head()

0        [displace, dog, jump, jeep, please, share, hel...
1        [water, seep, studio, buffalo, bayou, about, m...
2        [open, shelter, north, shore, grade, center, h...
3        [this, dog, walk, around, sinton, tx, carry, e...
4        [please, remember, harvey, still, active, stor...
5        [buffalo, bayou, come, studios, they, go, seco...
6        [harvey, at, least, two, people, have, be, kil...
7        [please, retweet, link, link, link, houston, h...
8        [please, retweet, link, link, link, houston, h...
9        [więcej, skutkach, huraganu, harvey, w, relacj...
10       [catastrophic, flood, underway, across, texas,...
11       [new, flash, flood, emergency, for, catastroph...
12       [harvey, major, flood, ongoing, harvey, barely...
13       [most, us, houston, ca, get, church, morning, ...
14       [a, tv, station, houston, khou, move, broadcas...
15       [there, several, days, continuous, rain, consi...
16       [dozens, shelter, open, along, path, hurricane.

In [66]:
dictionary = gensim.corpora.Dictionary(processed_tweets)
count = 0

#now show the most frequently-tweeted words
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

#now filter out words that show up too infrequently or too frequently
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

0 displace
1 displacedpets
2 dog
3 find
4 harvey
5 help
6 hurricane
7 jeep
8 jump
9 link
10 owner
